<a href="https://colab.research.google.com/github/vladimiralencar/Alunos-UEPB-TopicosEspeciaisEmBancoDeDados/blob/master/Copy_of_Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

|  |  |
|-------------|-------|
| 🎓 **Aprendizado** | Supervisionado |
| 📋 **Tarefa** | Classificação ou Regressão |
| 🔧 **Normalização** | Não |
| ⭐ **Dificuldade** | Médio |

# ⚙️ 0. Dependências

In [ ]:
import numpy as np
import pydotplus
from IPython.display import Image

from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.tree import (
    DecisionTreeClassifier,
    DecisionTreeRegressor,
    export_graphviz,
    export_text
)

# 🔍 1. Introdução

**Árvores de Decisão** são um dos algoritmos de Machine Learning mais utilizados. Principalmente por que são algoritmos de fácil interpretação e não precisam de normalização dos dados para funcionarem.

A ideia principal é dividir o problema em sub-problemas mais simples até que se resolva o problema. Nas árvores, cada **nó de decisão** contém um teste em um atributo, cada **folha** representa uma classe ou um valor (no caso da regressão) e o percurso da raiz até uma folha representa uma **regra de classificação/regressão**. Um atributo pode aparecer mais de uma vez na árvore, porém com valores diferentes.

**Algoritmo básico:**
1. Escolher um atributo
2. Dividir o (sub-)banco por uma valor específico do atributo
3. Para cada folha:

    3.1: Se todos os exemplos são da mesma classe, associar essa classe aos exemplos
    
    3.2: Caso contrário, repetir os passos 1 a 3

As **condições de paradas** podem ser inúmeras:
- Os atributos acabaram (no caso em que os atributos não se repetem na árvore)
- Todos os exemplos são de uma mesma classe
- A altura da árvore atingiu um valor previamente definido
- O número de exemplos a serem divididos é menor que um valor definido


✅ **Vantagens:**
- Fáceis de entender e explicar. Mais fácil inclusive que regressão linear
- Um dos poucos algoritmos que **não precisa de normalização**
- Algumas pessoas acreditam que ás arvores de decisão representam a tomada de decisão mais próxima dos seres humanos.
- Podem ser mostradas graficamente e facilmente interpretadas por não-especialistas


❌ **Desvantagens:**
- A precisão não é tão boa quanto outros algoritmos
- Não são robustas. Uma pequena mudança nos dados pode causar uma grande diferença na árvore final.


    
## Impureza e Ganho de Informação

*Como escolher o melhor atributo?* Existem muitas medidas e algoritmos diferentes:
- **ID3 e C4.5**: utilizam *ganho da informação*.
- **CART**: utiliza *impureza de Gini*.
- **CHAID**: utilizam significância estatística.

Em geral, todas as abordagens concordam em dois pontos:
- Uma divisão que mantém as proporções das classes é inútil
- Uma divisão onde todos os exemplos são da mesma classe, tem utilidade máxima

### Entropia
A **Entropia** caracteriza a impureza de uma coleção arbitrária de exemplos.

Seja $S$ uma amostra de exemplos e $p_i$ a probabilidade de cada classe $i$. A entropia $E(S)$ é definida como:

$$E(S) = \sum_i^n{p_i\ln{p_i}}$$

### Ganho de Informação

O **Ganho de Informação (GI)** é a diferença entre a impureza atual (entropia, gini, etc..) e a impureza ponderada dos dois novos grupos. Intuitivamente, o **GI representa a divisão que reduz a impureza, ou seja, obtém grupos mais homogêneos em comparação com o grupo antes da divisão**. Comparando o GI para várias divisões baseadas nas regras de divisão diferente nos permite escolher a "melhor" divisão.

# 🎲 2. Dados

| Tempo | Temperatura | Umidade | Vento |
|-------|-------------|---------|-------|
| Sol | 85 | 85 | Não |
| Sol | 80 | 90 | Sim |
| Nublado | 83 | 86 | Não |
| Chuva | 70 | 96 | Não |
| Chuva | 68 | 80 | Não |
| Chuva | 65 | 70 | Sim |
| Nublado | 64 | 65 | Sim |
| Sol | 72 | 95 | Não |
| Sol | 69 | 70 | Não |
| Chuva | 75 | 80 | Não |
| Sol | 75 | 70 | Sim |
| Nublado | 72 | 90 | Sim |
| Nublado | 81 | 75 | Não |
| Chuva | 71 | 91 | Sim |

In [ ]:
x = np.array(
    [
        [0, 85, 85, 0],
        [0, 80, 90, 1],
        [1, 83, 86, 0],
        [2, 70, 96, 0],
        [2, 68, 80, 0],
        [2, 65, 70, 1],
        [1, 64, 65, 1],
        [0, 72, 95, 0],
        [0, 69, 70, 0],
        [2, 75, 80, 0],
        [0, 75, 70, 1],
        [1, 72, 90, 1],
        [1, 81, 75, 0],
        [2, 71, 91, 1]
    ]
)

y = np.array([0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0]).reshape(-1, 1)

print(x.shape, y.shape)

(14, 4) (14, 1)


# 💻 3. Implementação

## Métricas de Impureza

In [ ]:
def entropy(y):
    if len(y) == 0:
        return 0.0

    _, class_counts = np.unique(y, return_counts=True)
    proportions = class_counts / y.shape[0]
    return -np.sum(proportions * np.log2(proportions))

def gini(y):
    if len(y) == 0: return 0.0

    _, class_counts = np.unique(y, return_counts=True)
    proportions = class_counts / y.shape[0]
    return 1.0 - np.sum(proportions**2)


def mse(y: np.ndarray):
    if len(y) == 0:
        return 0.0

    mean_y = np.mean(y)
    return np.mean((y - mean_y) ** 2)

In [ ]:
def tree_as_text(node, feature_names, indent=''):
    if node.is_leaf: return 'Leaf: y = {}\n'.format(node.output)

    is_discrete = isinstance(node.value, str)
    right_condition = f"{feature_names[node.col_index]} {'==' if is_discrete else '>='} {node.value} | "
    left_condition = indent
    left_condition += f"{feature_names[node.col_index]} {'!=' if is_discrete else '< '} {node.value} | "

    summary = right_condition + tree_as_text(node.right_child, feature_names, indent + ' '*len(right_condition))
    summary += left_condition + tree_as_text(node.left_child, feature_names, indent + ' '*len(right_condition))

    return summary

In [ ]:
class Node():
    def __init__(self, parent=None):
        self.parent = parent
        self.left_child = None   # less than a value (regression) OR not equal to a category (classification)
        self.right_child = None  # otherwise
        self.impurity = None
        self.col_index = None
        self.value = None
        self.is_leaf = True
        self.output = None


class BaseTree():
    def __init__(self, criterion=gini, max_depth=None, min_samples_split=2):
        self.criterion = criterion
        self.max_depth = np.inf if max_depth is None else max_depth
        self.min_samples_split = min_samples_split
        self.root = Node()

    def fit(self, x, y):
        self._build_tree(self.root, x, y)

    def predict(self, x):
        return np.array([self._predict_recursive(sample, self.root) for sample in x])

    def _compute_output(self, y):
        raise NotImplementedError()

    def _build_tree(self, parent_node, x, y, depth=0):
        parent_node.col_index, parent_node.value, parent_node.impurity = self._find_best_split(x, y)
        if (
            parent_node.impurity == 0
            or len(y) < self.min_samples_split
            or depth > self.max_depth
        ):
            parent_node.output = self._compute_output(y)
            return

        x_left, y_left, x_right, y_right = self._split_data(x, y, parent_node.col_index, parent_node.value)

        left_child = Node(parent=parent_node)
        self._build_tree(left_child, x_left, y_left, depth + 1)

        right_child = Node(parent=parent_node)
        self._build_tree(right_child, x_right, y_right, depth + 1)

        parent_node.is_leaf = False
        parent_node.left_child = left_child
        parent_node.right_child = right_child


    def _find_best_split(self, x, y):
        best_impurity, best_col, best_value = 0.0, None, None

        n_features = x.shape[1]
        for feature_idx in range(n_features):
            feature_values = np.unique(x[:, feature_idx])
            for value in feature_values:
                gain = self._information_gain(x, y, feature_idx, value)
                if gain > best_impurity:
                    best_impurity = gain
                    best_col = feature_idx
                    best_value = value

        return best_col, best_value, best_impurity

    def _information_gain(self, x, y, col_index, threshold):
        parent_impurity = self.criterion(y)

        _, y_left, _, y_right = self._split_data(x, y, col_index, threshold)
        if len(y_left) == 0 or len(y_right) == 0:
            return 0.0

        left_impurity = self.criterion(y_left)
        right_impurity = self.criterion(y_right)
        p = len(y_left) / y.shape[0]
        child_impurity = p * left_impurity + (1 - p) * right_impurity

        gain = parent_impurity - child_impurity
        return gain

    def _split_data(self, x, y, col_index, threshold):
        left_mask = x[:, col_index] < threshold
        right_mask = np.invert(left_mask)
        return x[left_mask], y[left_mask], x[right_mask], y[right_mask]

    def _predict_recursive(self, x, node):
        if node.is_leaf: return node.output

        right_condition = x[node.col_index] > node.value
        return self._predict_recursive(x, node.right_child if right_condition else node.left_child)


## 🏷️ Classificador

In [ ]:
class DTClassifier(BaseTree):
    def _compute_output(self, y):
        classes, class_counts = np.unique(y, return_counts=True)
        return classes[np.argmax(class_counts)]

In [ ]:
dt = DTClassifier(max_depth=3)
dt.fit(x, y)

y_pred = dt.predict(x)
print(y_pred)

[1 0 0 1 1 1 1 0 1 1 1 0 1 1]


### Comparação com o Scikit-learn

In [ ]:
x_clf, y_clf = make_classification(n_samples=1000, n_features=4, n_classes=2, n_redundant=0, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_clf, y_clf, test_size=0.3, random_state=42)

dt = DTClassifier(criterion=gini, max_depth=5, min_samples_split=2)
dt.fit(x_train, y_train)
accuracy = accuracy_score(y_test, dt.predict(x_test))
print(f"Accuracy: {accuracy:.3f}")

sk_dt = DecisionTreeClassifier(criterion='gini', max_depth=5, min_samples_split=2, random_state=42)
sk_dt.fit(x_train, y_train)
sk_accuracy = accuracy_score(y_test, sk_dt.predict(x_test))
print(f" Sklearn: {sk_accuracy:.3f}")

Accuracy: 0.910
 Sklearn: 0.910


## 📈 Regressor

In [ ]:
class DTRegressor(BaseTree):
    def _compute_output(self, y):
        return np.mean(y)

### Comparação com o Scikit-learn

In [ ]:
x_reg, y_reg = make_regression(n_samples=1000, n_features=4, n_targets=1, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_reg, y_reg, test_size=0.3, random_state=42)

dt = DTRegressor(criterion=mse, max_depth=5, min_samples_split=2)
dt.fit(x_train, y_train)
error = mean_squared_error(y_test, dt.predict(x_test))
print(f"MSE: {error:.3f}")

sk_dt = DecisionTreeRegressor(criterion='squared_error', max_depth=5, min_samples_split=2, random_state=42)
sk_dt.fit(x_train, y_train)
sk_error = mean_squared_error(y_test, sk_dt.predict(x_test))
print(f"MSE: {sk_error:.3f}")

MSE: 2369.947
MSE: 2627.595


# 💭 Considerações Finais

- A implementação do scikit-learn depende de um `random_state`. Logo, é difícil fazer todos os resultados baterem sempre.

**random_state**: Controls the randomness of the estimator. The features are always randomly permuted at each split, even if splitter is set to "best". When max_features < n_features, the algorithm will select max_features at random at each split before finding the best split among them. **But the best found split may vary across different runs, even if max_features=n_features**. That is the case, if the improvement of the criterion is identical for several splits and one split has to be selected at random. To obtain a deterministic behaviour during fitting, random_state has to be fixed to an integer. See Glossary for details.
